In [1]:
import pickle
import pandas as pd
import nltk
import re

In [2]:
model = pickle.load(open('./model.pkl', 'rb'))
countvect = pickle.load(open('./countvect.pkl', 'rb'))

/home/rohit/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/rohit/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
from nltk.corpus import stopwords

In [4]:
def clean_texts(texts):
    stwords = stopwords.words('english')
    l = len(texts)/10
    temp_texts = []
    for i in range(len(texts)):
        text = re.sub('\d','0',texts[i])
        if 'www.' in text or 'http:' in text or 'https:' in text or '.com' in text: # remove links and urls
            text = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", " ", text)
        
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = text.split()
        text = [word for word in text if not word in stwords] # remove stopwords 
        text = ' '.join(text)
        temp_texts.append(text)
        if i%l==0:
            print('--'+str(int(i/l)*10)+'%', end='')
    print('--100%--Done !')
    return temp_texts

In [5]:
from scrapy import AmazonReviews

In [6]:
obj = AmazonReviews("https://www.amazon.in/LG-Inverter-Frost-Free-Refrigerator-GL-I292RPZX/dp/B08X72GY5Q/ref=lp_41909197031_1_1")

 LG 260L 3 Star Smart Inverter Frost-Free Double Door Refrigerator (GL-I292RPZX, Shiny Steel, Door Cooling+)       


In [7]:
reviews = obj.scrap(pages=2)

In [8]:
print(reviews)

['Abhi to mast chal Raha hai baad me dekh te hai', 'Good for a 5 members family', 'LG was one of the trusted brand in electronic items, but this model is worst. Within 5 months it shows cooling issues. Only freezer is cooling up lower fridge storage not cooling, full of water drops falling from freezer bottom size.', "                    The media could not be loaded.                \xa0I received the product on time and LG demo was done on the same day of deilvery.PROS:-Fridge build and finish is good.-There is lot of space for storage.-It runs with smart inventor technology-has huge fruit and vegetables tray.-DOOR COOLING PLUS IS GREAT FEATURES AND USEFUL, WHICH GIVE COOLING FROM FRONT AND BACK.-Smart diagnosis (i haven't use this feature, but it useful to diagnosis any issue)-So overall this is excellent buy for my requirements and I can recommend this product.Few cons which I observed:-There is no light freezer compartment.-No separate tray or space for dairy productsThis 2 cons we

In [9]:
reviews_clean = clean_texts(reviews)
print(reviews_clean)

--0%--100%--Done !
['abhi mast chal raha hai baad dekh te hai', 'good members family', 'lg one trusted brand electronic items model worst within months shows cooling issues freezer cooling lower fridge storage cooling full water drops falling freezer bottom size', 'media could loaded received product time lg demo done day deilvery pros fridge build finish good lot space storage runs smart inventor technology huge fruit vegetables tray door cooling plus great features useful give cooling front back smart diagnosis use feature useful diagnosis issue overall excellent buy requirements recommend product cons observed light freezer compartment separate tray space dairy productsthis cons dealer break', 'good', 'nice product timely delivery', 'side looking superb inside plastic material delicate compare old lg refrigerator overall experience ok', 'good product delivered', 'nice product good quality', 'good', 'lg product always good', 'good', 'bad post sell service', 'good product', 'fast cool

In [11]:
reviews_vect = countvect.transform(reviews_clean)

In [12]:
pred = model.predict(reviews_vect)

In [14]:
df = pd.DataFrame({'Reviews':reviews,'Sentiment':pred})

In [15]:
htm = df.to_html()

In [16]:
htm

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Reviews</th>\n      <th>Sentiment</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Abhi to mast chal Raha hai baad me dekh te hai</td>\n      <td>0</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Good for a 5 members family</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>LG was one of the trusted brand in electronic items, but this model is worst. Within 5 months it shows cooling issues. Only freezer is cooling up lower fridge storage not cooling, full of water drops falling from freezer bottom size.</td>\n      <td>0</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>The media could not be loaded.                \xa0I received the product on time and LG demo was done on the same day of deilvery.PROS:-Fridge build and finish is good.-There is lot of space for storage.-It runs with smart inventor technology-has

In [18]:
from flask import Flask, render_template_string

In [21]:
render_template_string("<html>hello worlds</html>")

AttributeError: 'NoneType' object has no attribute 'app'

In [13]:
import requests
from bs4 import BeautifulSoup

In [22]:
url = "https://www.amazon.in/Apple-iPhone-128GB-Product-RED/dp/B0BDJVSDMY/ref=sr_1_omk_4?crid=139UH08QB3SG1&keywords=iphone+13&qid=1669532415&qu=eyJxc2MiOiI0LjYyIiwicXNhIjoiNC4wNyIsInFzcCI6IjMuNDcifQ%3D%3D&sprefix=iphone%2Caps%2C268&sr=8-4"
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36','referer':url}

In [23]:
page = requests.get(url, headers=header)

In [24]:
soup = BeautifulSoup(page.content, features="html.parser")

In [29]:
title = soup.findAll("span", {"class": "a-size-large product-title-word-break"})[0]#.text[7:]

In [30]:
title

<span class="a-size-large product-title-word-break" id="productTitle">        Apple iPhone 14 128GB (Product) RED       </span>